In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#data.drop("RowNumber", axis=1, inplace=True)
#data.drop("CustomerId", axis=1, inplace=True)
#data.drop("Surname", axis=1, inplace=True)
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#X=data.iloc[:,0:10].values
#Y=data.iloc[:,-1].values

#dataset = pd.read_csv('Churn_Modelling.csv')
X = data.iloc[:, 3:-1].values
y = data.iloc[:, -1].values

In [5]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,2]=le.fit_transform(X[:,2])


In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[("encode",OneHotEncoder(),[1])],remainder='passthrough')
X=np.array(ct.fit_transform(X))
print(X)


[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)



# Initialize ANN

In [9]:
ann=tf.keras.models.Sequential()
#adding input layer and first hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))
#adding the second hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))
#adding the output layer
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

## Compiling the ANN

In [10]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

## Trainig the ANN model on training set

In [11]:
#neural network has to be trained over a certain epoches to improve the accuracy over time
ann.fit(X_train,Y_train,batch_size=32,epochs=100)

Epoch 1/100


250/250 [==============================] - 1s 1ms/step - loss: 0.7491 - accuracy: 0.4891
Epoch 2/100
250/250 [==============================] - 0s 942us/step - loss: 0.5198 - accuracy: 0.7944
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4612 - accuracy: 0.8092
Epoch 4/100
250/250 [==============================] - 0s 963us/step - loss: 0.4386 - accuracy: 0.8164
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4260 - accuracy: 0.8173
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4144 - accuracy: 0.8194
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4011 - accuracy: 0.8256
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3879 - accuracy: 0.8321
Epoch 9/100
250/250 [==============================] - 0s 992us/step - loss: 0.3764 - accuracy: 0.8396
Epoch 10/100
250/250 [==============================] - 0s 967us/step - loss: 0.368

## Making the predictions and evaluating the model

In [12]:
Value=ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))
Value=Value[0][0]
if Value < 0.5:
    print('customer will leave')
else:
    print('customer stay')

1/1 [==============================] - 0s 98ms/step
customer will leave


## predicting the test set result

In [13]:
y_pred=ann.predict(X_test)
y_pred=(y_pred>0.5)
print(y_pred)

63/63 [==============================] - 0s 2ms/step
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


In [14]:
print(np.concatenate((y_pred.reshape(len(y_pred),1), Y_test.reshape(len(Y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Making a confusion matrix

In [15]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(Y_test,y_pred)
print(cm)
accuracy_score(Y_test,y_pred)

[[1528   67]
 [ 203  202]]


0.865